# Convolutional Neural Network in `tflearn`

## Import dependencies

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

## Data Loading

In [ ]:
# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data('../datasets/MNIST/', one_hot=True)

### Data preprocessing

In [ ]:
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

## Building the Convolutional Neural Network

In [ ]:
# Input layer
network = input_data(shape=[None, 28, 28, 1], name='input')

# 1st convolutional block
network = conv_2d(network, 32, 3, activation='relu', regularizer='l2')
network = max_pool_2d(network, 2)
network = local_response_normalization(network)

# 2nd convolutional block
network = conv_2d(network, 64, 3, activation='relu', regularizer='l2')
network = max_pool_2d(network, 2)
network = local_response_normalization(network)

# 1st Fully connected layer
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)

# 2nd Fully connected layer
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)

# 3rd fully connected layer
network = fully_connected(network, 10, activation='softmax')

# Regression/Classification layer
network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

## Running the convnet

In [ ]:
# Training
model = tflearn.DNN(network, tensorboard_verbose=0, tensorboard_dir='../saved/convnet/tflearn/tensorboard')
model.fit({'input': X}, {'target': Y}, n_epoch=10, batch_size=52,
          validation_set=({'input': testX}, {'target': testY}),
          validation_batch_size=20, snapshot_step=100,
          show_metric=True, run_id='convnet_mnist')